In [1]:
import sys, os
sys.path.append('/Users/pculviner/notebook/bin/mtbrnaseq/')
from mtbrnaseq import *
from dask.distributed import LocalCluster

env = os.environ
conda_path = os.path.dirname(sys.executable)  # store conda path for shell execution
env['PATH'] = conda_path + os.pathsep + env['PATH']

In [2]:
client = LocalCluster(
    n_workers=3,
    threads_per_worker=1).get_client()
demultiplexDatasets(
   'validation_fastqs.csv', 'validation_inline.csv', '../rnaseq/fastqs', '../rnaseq/validation',
    client=client, threads=5)
client.shutdown()

Demultiplexing from input fastqs....


100%|███████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.14s/it]


Trimming fastqs....


100%|█████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:11<00:00,  2.36it/s]


Concatenating outputs....


100%|███████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 3050.81it/s]


Cleaining up....


In [3]:
client = LocalCluster(
    n_workers=4,
    threads_per_worker=1,).get_client()
mapSplitFastqs(
    'validation_inline.csv', '../rnaseq/validation/split_fastqs', '../rnaseq/validation/',  '../datasets/Mtb_h37rv.fasta',
    rm_duplicates=True, threads=5, client=client)
client.shutdown()

Preparing index....
Mapping samples....


100%|█████████████████████████████████████████████████████████████████████████████████████████| 27/27 [05:19<00:00, 11.82s/it]


Parsing outputs....


100%|███████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 5720.08it/s]


Cleaning up....


In [4]:
client = LocalCluster(
    n_workers=15,
    threads_per_worker=1,).get_client()
counts, statistics = bamsToGeneCounts(
    'validation_inline.csv', '../rnaseq/validation/bams', '../datasets/gt_mtb_h37rv.csv',
    client=client, mapq_cutoff=10)
client.shutdown()

Mapping fragments to regions....


100%|█████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 17.25it/s]


In [6]:
counts.to_csv('../rnaseq/validation/validation_counts.csv')
statistics.to_csv('../rnaseq/validation/region_summary.csv')